In [1]:
import pandas as pd

In [2]:
# make the dictionary whose keys are file names and values are 1-dimensional ndarray.
# we don't need teampping here because it doesn't have "Tournament" feature.
data_files = ["draft_phase", "eco_rounds", "eco_stats", "kills_stats",
              "kills", "maps_played", "maps_scores", "overview", "rounds_kills",
              "scores", "win_loss_methods_count", "win_loss_methods_round_number"]
tournaments = {}
for file in data_files:
    filename = f"../data/vct_2022/matches/{file}.csv"
    tournaments[file] = pd.read_csv(filename).Tournament.unique()

In [3]:
print("The number of unique tournaments of each file.")
for file, df in tournaments.items():
    print(f"{file}: {len(df)}")

The number of unique tournaments of each file.
draft_phase: 53
eco_rounds: 58
eco_stats: 58
kills_stats: 58
kills: 58
maps_played: 58
maps_scores: 58
overview: 58
rounds_kills: 58
scores: 58
win_loss_methods_count: 58
win_loss_methods_round_number: 58


In [4]:
files_by_tnmt_len = {}

for file, df in tournaments.items():
    if len(df) in files_by_tnmt_len:
        files_by_tnmt_len[len(df)].append(file)
    else:
        files_by_tnmt_len[len(df)] = [file]

files_by_tnmt_len

{53: ['draft_phase'],
 58: ['eco_rounds',
  'eco_stats',
  'kills_stats',
  'kills',
  'maps_played',
  'maps_scores',
  'overview',
  'rounds_kills',
  'scores',
  'win_loss_methods_count',
  'win_loss_methods_round_number']}

In [5]:
def not_in_the_other(a: list, b: list) -> list: 
    '''
    input: a pair of lists
    return: the list of elements which are in the list a, but not in b.
    '''
    not_in_b = []
    for i in a:
        if i not in b:
            not_in_b.append(i)
            
    return not_in_b

def all_pairs(k: int) -> list:
    '''
    input: a postive integer k bigger than 1
    return: list of all pairs from 0,1,2,...,k-1.  Two elements in a pair are different.
    '''
    if k < 2:
        return print("input needs to be an integer bigger than 1")
    
    else:
        pairs = []
        for i in range(k):
            n = i
            m = i+1
            while m < k:
                pairs.append((n,m))
                m += 1
        return pairs

In [6]:
for length, file_list in files_by_tnmt_len.items():
    print(f"Check if all the files, having {length} many tournaments, share the same tournaments.")
    if len(file_list) == 1:
        print(f"There is only one file with {length} many tournaments.")
    else:
        pair_ind = all_pairs(len(file_list))
        diff_pair_ind = []

        for i, j in pair_ind:
            not_in_intersection = not_in_the_other(tournaments[file_list[i]], tournaments[file_list[j]])
            if not_in_intersection != []:
                print(not_in_intersection)
                diff_pair_ind.append((i, j))

        if len(diff_pair_ind) > 0:
            for i, j in diff_pair_ind:
                print(f"{file_list[i]} and {file_list[j]} have different tournaments.")
        else:
            print("All files have the same tournaments as we want.")
    print("====================")

Check if all the files, having 53 many tournaments, share the same tournaments.
There is only one file with 53 many tournaments.
Check if all the files, having 58 many tournaments, share the same tournaments.
All files have the same tournaments as we want.


In [7]:
### Is tournaments["draft_phase"] a subset of tournaments["eco_rounds"]?
sample_file_names = ["draft_phase", "eco_rounds"]
sample_dict = {}
pairs_in_smpl_dict = all_pairs(len(sample_file_names))

for i, j in pairs_in_smpl_dict:
    missing_tnmt = not_in_the_other(tournaments[sample_file_names[i]], tournaments[sample_file_names[j]])
    if missing_tnmt:
        print(missing_tnmt, f"is in {sample_file_names[i]}, but not in {sample_file_names[j]}")
    else:
        print(f"All the tournaments in {sample_file_names[i]} are also in {sample_file_names[j]}")

All the tournaments in draft_phase are also in eco_rounds


##### I would like to check if missing tournaments are important ones:

In [8]:
not_in_the_other(tournaments["eco_rounds"], tournaments["draft_phase"])

['Road to VCT 2022: Stage 2',
 'Champions Tour Latin America South Stage 1: Challengers',
 'Road to VCT 2022',
 'Champions Tour Turkey Stage 1: Challengers 1',
 'Champions Tour CIS Stage 1: Challengers 1']